<a href="https://colab.research.google.com/github/sophielu05/114-1-/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install -q google-generativeai

In [34]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [35]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('lty')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [36]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1dOIXC4DnmBN-qyEXa7g-mpzmZZ-CAWjaFXdNpiYfclA/edit?usp=sharing"
WORKSHEET_NAME = "工作表1"

REQUIRED_COLUMNS = ["日期", "科目", "考試成績"]

_auth_done = False
_gc = None
_ws = None

In [37]:
import gspread
from google.colab import auth
import google.auth
from datetime import datetime
import gradio as gr
import pandas as pd
import json

# --- 您的 Google Sheet 設定 (請修改成您自己的) ---
# Google Sheet 的完整 URL (請替換為您的 Sheet URL)
SHEET_URL = "https://docs.google.com/spreadsheets/d/1vDINC4DxmBN-qYEXa7g-mprzZZ-CAWjAFXdNWpiYfCla/edit?usp=sharing"
# 工作表名稱
WORKSHEET_NAME = "工作表1"
# 預期的欄位名稱
REQUIRED_COLUMNS = ["日期", "科目", "考試成績"]
# ---------------------------------------------------

# 為了讓 Gradio 介面運行，我們將身份驗證移到外部
# 並且在首次執行函數時，才進行 gspread 的授權連線。
gc = None

# --- 1. 身份驗證與授權 ---
def authorize_gspread():
    """執行 Google Colab 身份驗證並取得 gspread 連線物件。"""
    global gc
    if gc is None:
        try:
            # Colab 身份驗證
            print("正在進行 Google 身份驗證...")
            # 執行後會跳出視窗要求您登入 Google 帳號，並授權 Colab 存取 Google Drive/Sheets
            auth.authenticate_user()

            # 使用 google.auth.default()[0] 取得正確的憑證物件 (解決 tuple error)
            creds, _ = google.auth.default()
            gc = gspread.authorize(creds)
            print("✅ 身份驗證完成。")
            return gc
        except Exception as e:
            print(f"❌ 身份驗證或授權失敗: {e}")
            return None
    return gc


# --- 2. 核心功能：記錄成績 (供 Gradio 調用) ---
def record_score_to_sheet(subject, score_input):
    """
    將一筆新的成績紀錄附加到 Google Sheet 的下一行。
    :param subject: 科目名稱 (str)
    :param score_input: 成績 (str，因為 Gradio 輸入框是字串)
    """
    gc = authorize_gspread()
    if gc is None:
        return "❌ 錯誤：無法連接到 Google Sheet，請檢查 Colab 授權步驟是否完成。"

    try:
        # 嘗試將分數輸入轉換為數字，以便 Google Sheet 格式化
        try:
            score = float(score_input)
        except ValueError:
            score = score_input # 如果不是數字，則以原始字串記錄

        # 根據 URL 開啟工作簿 (Spreadsheet)
        spreadsheet = gc.open_by_url(SHEET_URL)

        # 選擇工作表
        worksheet = spreadsheet.worksheet(WORKSHEET_NAME)

        # 取得當前日期 (YYYY-MM-DD)
        date_str = datetime.now().strftime('%Y-%m-%d')

        # 準備要寫入的資料：[日期, 科目, 考試成績]
        new_row_data = [date_str, subject, score]

        # 確保工作表有標題列
        current_header = worksheet.row_values(1)
        if current_header != REQUIRED_COLUMNS:
             worksheet.update('A1:C1', [REQUIRED_COLUMNS])

        # 附加新的資料行
        worksheet.append_row(new_row_data, value_input_option='USER_ENTERED')

        # 返回成功訊息給 Gradio 介面
        return f"✅ 成功記錄成績！\n日期: {date_str}, 科目: {subject}, 成績: {score}\n資料已寫入 '{WORKSHEET_NAME}'。"

    except gspread.exceptions.SpreadsheetNotFound:
        return f"❌ 錯誤：找不到 Google Sheet，請檢查 URL 是否正確 ({SHEET_URL})。"
    except gspread.exceptions.WorksheetNotFound:
        return f"❌ 錯誤：找不到工作表 '{WORKSHEET_NAME}'，請檢查名稱是否正確。"
    except Exception as e:
        return f"❌ 記錄時發生未知錯誤: {e}"


# --- 3. Gradio 介面設置 ---

# 介面標題
title = "Google Sheet 成績記錄器"

# 介面描述
description = f"""
使用 Google Colab 和 Gradio 快速記錄成績到 Google Sheet。
**目標 Sheet:** {SHEET_URL}
**目標工作表:** {WORKSHEET_NAME}
"""

# 創建 Gradio 介面
iface = gr.Interface(
    # 綁定核心函數
    fn=record_score_to_sheet,

    # 輸入組件：科目 (文字框), 成績 (數字框)
    inputs=[
        gr.Textbox(label="科目名稱", placeholder="例如：數學、英文"),
        gr.Number(label="考試成績", minimum=0, maximum=100)
    ],

    # 輸出組件：顯示結果訊息
    outputs=gr.Textbox(label="記錄結果", lines=5),

    # 介面標題與描述
    title=title,
    description=description,

    # 讓按鈕的文字更清楚
    submit_btn="記錄成績到 Google Sheet",

    # 禁用清除按鈕，避免誤觸
    clear_btn=None
)

# 啟動 Gradio 介面
print("\n--- 正在啟動 Gradio 介面 ---")
print("請點擊生成的 Public URL 連結在瀏覽器中使用。")

# Launch 參數
# share=True 會生成一個外部連結，方便在瀏覽器中開啟
iface.launch(share=True)


--- 正在啟動 Gradio 介面 ---
請點擊生成的 Public URL 連結在瀏覽器中使用。
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d70e1e0318dae8249c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
